# Predicting the scale price of Bulldozers using Machine Learning Model

* In this notebook, we are going to go through an example of machine learning project with the goal of predicting the scale price of bulldozers.
### 1. Problem Definition
* How well can we predict the future sale price of bulldozer, given its characteristics & previous examples of how much similar bulldozer have sold for?
### 2. Data 
* The data is downloaded from the Kaggle Blue book for Bulldozer competition.
##### https://www.kaggle.com/competitions/bluebook-for-bulldozers/data
* There are 3 main Types of data:
    * Train.csv is the training set, which contains data through the end of 2011.
    * Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
    * Test.csv is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition
### 3. Evaluation 
* The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.
##### https://www.kaggle.com/competitions/bluebook-for-bulldozers/overview
* The goal of most regression evaluation metrics is to minimize the error.
### 4. Features 
Kaggle provides a data dictionary detailing all the feature of the dataset.
     
